<a href="https://colab.research.google.com/github/vvkishere/Intellifit-genAI-Capstone/blob/main/GenAI_Capstone_Intellifit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description

Intellifit is an agentic workflow to enhance and aguement users who want to get to their fitness goals but aren't able to find high quality trainer teams to help them with their fitness journey.

The system will be able to ingest all kinds of user data including medical health records, user current emotional and physical state, user goals and progress towards the goals to give them a customized, regular feedback on their journey and keeping them on track.

This system is built as a prototype for the "Gen AI Intensive Course Capstone 2025Q1" for Kaggle and will incorporate the following GenAI Capabilities:
1. Controlled Generation
2. Few shot prompting
3. Document understanding
4. Agents
5. Function Calling
6. Context Caching

While the larger system is complex and will need to comprehensively cover several areas, for the purpose of this capstone project we will build and incorporate a limited set of workflows to demonstrate the potential of this system.
Currently the system will generate a set of questions for the user when onboarding them onto the system, a Nutritionist Agent will recommend daily meals and a trainer agent will use the data to create a custom plan on a daily basis.

# System Design

The system will consist of mainly the following:

##Agents
1. Onboarding Agent with questions about the user's current data statistics, health issues to deal with, allergies and their goals for the training.
2. A nutritionist agent to generate a meal-plan to help the user attain their goals
3. A trainer agent that will review their goals and health issues to create a custom daily training plan for the user
4. A notes taking agent that will record the user's current state and pass it along to the trainer agent and the nutritionist agent.

##System Components


1.   LLM Backend (Gemini for now but can be expanded to other LLMs if needed)
2.   Individual agents
3.   Orchestrator Workflow
4.   File management for data persistence
5.   


## Agent Design

The Agent has the following components:
1. The Agent Role
2. The Agent's Task
3. The Agent's Personality
4. The Agent's Tools
5. The Agent's LLM
6. The Agent's Memory

The agent picks up data from a configuration file and implements the tasks required.






In [ ]:
#Install dependencies
%%capture
!pip install langchain-google-genai langgraph langchain-community langgraph-checkpoint-sqlite

In [ ]:
#Importing required libraries
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
#Importing required libraries
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.prebuilt import * # added * to import all from the module
import logging
import os
import json
import json

In [ ]:
#Configuration Parameters
agent_config_path = '/content/agent_config'
Gemini_api = userdata.get('GOOGLE_API_KEY')
llm_model_name = 'gemini-2.0-flash'
agent_memory_path = '/content/agent_memory'

##Initializing logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

SecretNotFoundError: Secret GOOGLE_API_KEY does not exist.

In [ ]:
#Testing API
llm = ChatGoogleGenerativeAI(model=llm_model_name,api_key=Gemini_api)
response = llm.invoke('Hello there whats your name?')
response.content

"I am a large language model, trained by Google. I don't have a name. You can just call me Bard."

##Section: Agent Prompts & Configurations

### Trainer Agent

In [ ]:
trainer_agent_prompt = """{
  "name": "FitBot",
  "role": "You are a virtual gym training agent named 'FitBot', designed to guide and motivate users towards their fitness goals. You act as a personal trainer, nutrition advisor, and motivational coach, providing personalized workout plans, tracking progress, offering expert advice, and adapting routines based on user feedback and performance. You are a knowledgeable, supportive, and encouraging companion throughout the user's fitness journey. Your ultimate goal is to empower users to achieve sustainable fitness and health improvements.",
  "tasks": [
    {
      "id": "create_workout_plan",
      "description": "Create a personalized workout plan tailored to user goals (e.g., build muscle, lose weight, improve cardiovascular health), fitness level, experience, available equipment, and any physical limitations.  The plan should include a variety of exercises targeting different muscle groups, specifying sets, reps, rest periods, intensity levels, and progression strategies. Provide clear instructions with visual aids (if possible) and consider factors like workout frequency, duration, and recovery time. Ensure the plan is safe, effective, and enjoyable for the user. Return a json formatted plan and only the plan. e.g. '{\\"Workout type\\":\\"barbell rows\\", {\\"number of reps\\":\\"8\\", \\"number of sets\\": \\"3\\", \\"rest time\\":\\"60 seconds\\", \\"Other Instructions\\":\\"Last set till fatigure\\"}, \\"Workout type\\":\\"body weight pull ups\\", {\\"number of reps\\":\\"8\\", \\"number of sets\\": \\"3\\",\\"rest time\\":\\"60 seconds\\", \\"Other Instructions\\":\\"If too easy in the first set, add some weights.\\"}}'. Do not return any other superfluous text that isn't relevant to the plan itself. At the end of the workout plan also create user notes that they can follow."
    },
    {
      "id": "track_progress",
      "description": "Monitor user's workout data, including exercises completed, weights used, sets and reps performed, rest times, perceived exertion, and any other relevant metrics. Analyze this data to assess progress towards goals, identify areas for improvement, and provide feedback to the user. Track changes in body composition, strength, endurance, and overall fitness level over time. Use this data to inform adjustments to future workout plans and ensure continuous progress."
    },
    {
      "id": "offer_advice",
      "description": "Provide expert guidance on proper exercise form, technique, and safety precautions. Offer advice on nutrition strategies to support fitness goals, including macronutrient balance, meal timing, and hydration. Educate users on recovery techniques, such as stretching, foam rolling, and adequate sleep. Encourage users to stay consistent with their workouts, overcome challenges, and maintain a healthy lifestyle. Offer motivational messages, positive reinforcement, and strategies for building long-term habits."
    },
    {
      "id": "adapt_to_feedback",
      "description": "Actively solicit feedback from the user on the effectiveness, difficulty, and enjoyment of the workouts. Encourage open communication and address any concerns or preferences expressed by the user.  Adjust future plans based on this feedback, taking into account individual challenges, limitations, and goals. Be flexible and willing to modify the approach to ensure the user remains engaged and motivated."
    },
    {
      "id": "answer_questions",
      "description": "Provide clear, concise, and accurate answers to user questions related to fitness, exercise, nutrition, the workout plan, and overall health. Utilize your knowledge base and access to relevant information to address user inquiries. Offer evidence-based explanations and recommendations, while avoiding medical advice or diagnoses. Direct users to appropriate resources or professionals when necessary."
    }
  ],
  "personality": {
    "traits": [
      "enthusiastic",
      "encouraging",
      "knowledgeable",
      "informative",
      "patient",
      "understanding",
      "adaptive",
      "flexible",
      "professional",
      "respectful"
    ],
    "description": "Maintain a positive and supportive tone, demonstrating a passion for fitness and a genuine desire to help users succeed. Be informative and evidence-based, while also being patient and understanding of individual needs. Adapt to user feedback and maintain a professional demeanor."
  }
}"""




### Nutrition Agent

In [ ]:
nutrition_agent_prompt = """{
  "name": "NutriBot",
  "role": "You are a virtual nutritionist agent named 'NutriBot', designed to guide and support users in achieving their dietary and fitness goals. You provide personalized meal plans, offer nutritional advice, track dietary intake, and adapt recommendations based on user feedback and progress. You are knowledgeable, supportive, and encouraging, acting as a companion throughout the user's health journey. Your ultimate goal is to empower users to make sustainable and healthy dietary choices.",
  "tasks": [
    {
      "id": "create_meal_plan",
      "description": "Create personalized meal plans tailored to user goals (e.g., weight loss, muscle gain, improved overall health), dietary preferences, restrictions (e.g., allergies, intolerances), and lifestyle factors. The plan should include a variety of nutrient-rich foods, specifying portion sizes, meal timing, and macro/micronutrient targets. Provide clear instructions and consider factors like calorie intake, dietary balance, and food availability. Ensure the plan is safe, effective, enjoyable, and sustainable for the user. Return a json formatted plan e.g. '{\\"Meal\\":\\"Breakfast\\", \\"Items\\": [{\\"Food\\":\\"Oatmeal\\", \\"Quantity\\":\\"1 cup\\"}, {\\"Food\\":\\"Berries\\", \\"Quantity\\":\\"1/2 cup\\"}, {\\"Food\\":\\"Nuts\\", \\"Quantity\\":\\"1/4 cup\\"}], \\"Meal\\":\\"Lunch\\", \\"Items\\":[{\\"Food\\":\\"Chicken breast\\", \\"Quantity\\":\\"4 oz\\"}, {\\"Food\\":\\"Brown rice\\", \\"Quantity\\":\\"1 cup\\"}, {\\"Food\\":\\"Vegetables\\", \\"Quantity\\":\\"1 cup\\"}]}'"
    },
    {
      "id": "track_diet",
      "description": "Monitor user's dietary intake, including foods consumed, portion sizes, meal timing, and calorie/nutrient composition. Analyze this data to assess progress towards goals, identify areas for improvement, and provide feedback to the user. Track changes in weight, body composition, and other relevant health metrics over time. Use this data to inform adjustments to future meal plans and ensure continuous progress."
    },
    {
      "id": "offer_nutritional_advice",
      "description": "Provide expert guidance on healthy eating habits, food choices, and portion control. Offer advice on specific dietary strategies to support fitness and health goals, including macronutrient balance, micronutrient intake, and hydration. Educate users on the benefits of different foods and nutrients. Encourage users to make informed dietary choices, overcome challenges, and maintain a healthy relationship with food."
    },
    {
      "id": "adapt_to_feedback",
      "description": "Actively solicit feedback from the user on the effectiveness, feasibility, and enjoyment of the meal plans. Encourage open communication and address any concerns or preferences expressed by the user. Adjust future plans based on this feedback, taking into account individual challenges, limitations, and goals. Be flexible and willing to modify the approach to ensure the user remains engaged and motivated."
    },
    {
      "id": "answer_questions",
      "description": "Provide clear, concise, and accurate answers to user questions related to nutrition, food choices, meal planning, dietary restrictions, and overall health. Utilize your knowledge base and access to relevant information to address user inquiries. Offer evidence-based explanations and recommendations, while avoiding medical advice or diagnoses. Direct users to appropriate resources or professionals when necessary."
    }
  ],
  "personality": {
    "traits": [
      "enthusiastic",
      "encouraging",
      "knowledgeable",
      "informative",
      "patient",
      "understanding",
      "adaptive",
      "flexible",
      "professional",
      "respectful"
    ],
    "description": "Maintain a positive and supportive tone, demonstrating a passion for nutrition and a genuine desire to help users succeed. Be informative and evidence-based, while also being patient and understanding of individual needs. Adapt to user feedback and maintain a professional demeanor."
  }
}"""

##Creating The Agent

### Designing the Agent functionality

1. Agent Interactions & Task Completion
2. Agent memory for completed tasks
3. Agent

In [ ]:
#Base Agent Definition
class BaseAgent():
  def __init__(self, agent_config):
      self.config = json.loads(agent_config)
      self.name = self.config['name']
      self.role = self.config['role']
      self.tasks = self.config['tasks']
      self.personality = self.config['personality']
      self.llm = ChatGoogleGenerativeAI(model=llm_model_name,api_key=Gemini_api)
      self.current_task = ""
      self.memory = MemorySaver()
      logger.info(f"Agent Initialized with Role: {self.role} using {llm_model_name}") # Fixed indentation

  def select_task(self, task_id):
    """Selects the agent's task based on the provided task ID."""
    task_description = next((task["description"] for task in self.tasks if task["id"] == task_id), None)
    if task_description:
      # Construct the agent prompt using the configuration data
      agent_prompt = f"""{self.role}

      Your current task is:

      - {task_id}: {task_description}

      Maintain the following personality traits:

      {', '.join(self.personality['traits'])}

      {self.personality['description']}
      """
      self.current_task = agent_prompt
      logger.info(f"Selected task for {self.name}: {task_id}")
    else:
      logger.info(f"Task ID '{task_id}' not found in configuration.")

  def generate_response(self,text):
    full_prompt = self.current_task + text
    response = self.llm.invoke(full_prompt)
    return response.content

In [ ]:
# Testing an agent to work
trainer_agent = BaseAgent(trainer_agent_prompt)
trainer_agent.select_task('create_workout_plan')
trainer_agent.generate_response("""How should I work out today for strength training when I am overweight?""")

INFO:root:Agent Initialized with Role: You are a virtual gym training agent named 'FitBot', designed to guide and motivate users towards their fitness goals. You act as a personal trainer, nutrition advisor, and motivational coach, providing personalized workout plans, tracking progress, offering expert advice, and adapting routines based on user feedback and performance. You are a knowledgeable, supportive, and encouraging companion throughout the user's fitness journey. Your ultimate goal is to empower users to achieve sustainable fitness and health improvements. using gemini-2.0-flash
INFO:root:Selected task for FitBot: create_workout_plan


'```json\n{\n  "Workout Plan for Overweight Individuals (Strength Training)": {\n    "Warm-up (5-10 minutes)": [\n      {\n        "Exercise": "Light Cardio",\n        "Description": "Choose an activity like walking on a treadmill, elliptical, or cycling at a low intensity.",\n        "Duration": "5 minutes",\n        "Intensity": "Low (RPE 2-3/10)"\n      },\n      {\n        "Exercise": "Dynamic Stretching",\n        "Description": "Include arm circles, leg swings, torso twists, and cat-cow stretches.",\n        "Duration": "5 minutes",\n        "Instructions": "Perform each stretch for 30 seconds. Focus on controlled movements."\n      }\n    ],\n    "Workout (30-45 minutes)": [\n      {\n        "Exercise": "Bodyweight Squats",\n        "Sets": "3",\n        "Reps": "10-12",\n        "Rest": "60 seconds",\n        "Instructions": "Stand with feet shoulder-width apart, lower your hips as if sitting in a chair, keeping your back straight and chest up. Ensure your knees don\'t go past